In [1]:
from SeismTool.HSR import hsr
from imp import reload
from obspy import UTCDateTime,read
from SeismTool.io import seism,dataLib
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
reload(dataLib)
reload(seism)
#stations =  seism.StationList('hsrStd')
stations =  seism.StationList(seism.StationList('../stations/201805DXSel.txt'))
stations.set('net','*')
stations.set('compBase','?H')
stations.set('nameMode','201805DS')
plt.close()
plt.figure(figsize=[25,20])
for station in stations:
    plt.text(station['lo'],station['la'],station['sta'])
plt.xlim([115.65,115.90])
plt.ylim([39.06,39.22])
plt.savefig('hsrFig/201805Sta.jpg',dpi=200)
#plt.show()

In [2]:

T0=UTCDateTime(2018,5,10).timestamp
T3L = [seism.Trace3(seism.getTrace3ByFileName(station.\
    getFileNames(int(T0/86400)*86400,int(T0/86400)*86400),freq=[0.5, 40],delta0=0.01)) for station in stations]
print(stations[1],len(stations))
reload(seism)
reload(hsr)
#T3L =[seism.Trace3(t3)for t3 in T3L]
#x3 = T3L[0].cross(T3L[1].slice(T0,T0+3))


#####not too many noise std : 68539.492990#####
#####not too many noise std : 51488.013042#####
#####not too many noise std : 154312.463175#####
#####not too many noise std : 156131.872974#####
#####not too many noise std : 155593.905239#####
#####not too many noise std : 26736.287509#####
#####not too many noise std : 69642.552897#####
#####not too many noise std : 55281.440541#####
#####not too many noise std : 64270.940598#####
#####not too many noise std : 150221.476867#####
#####not too many noise std : 118741.529397#####
#####not too many noise std : 138456.449349#####
#####not too many noise std : 19237.890374#####
#####not too many noise std : 25297.450235#####
#####not too many noise std : 14550.917266#####
#####not too many noise std : 25948.856685#####
#####not too many noise std : 47195.538848#####
#####not too many noise std : 12278.953736#####
#####not too many noise std : 216555.267005#####
#####not too many noise std : 198463.606012#####
#####not too many noise std : 10

<module 'SeismTool.HSR.hsr' from '/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/SeismTool/HSR/hsr.py'>

In [3]:
h=hsr.hsr()
eL=h.getCatolog(T3L,sta0='PKU28',sta1='SUS47',stations=stations)

23.75 171 119


In [36]:
time0,time1,v0,v1=eL[0]
time0,time1=[min([time0,time1]),max([time0,time1])]
print(time0,time1,time1-time0)
t3L= [  T3.slice(time1+30,time1+60) for T3 in T3L]
for t3 in t3L:
    if t3.Data().size>0:
        t3.rotate(25)

1525913602.22 1525913614.68 12.4549918175


In [37]:
reload(hsr)
h= hsr.hsr()
specL  = h.getSpecL(t3L,1)
specLAbs=np.abs(specL,isNorm=False)
specLAbs/=specLAbs.std(axis=1,keepdims=True)
plt.pcolormesh(specLAbs,rasterized=True)
plt.savefig('hsrFig/specL.jpg',dpi=300)

/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [38]:
plt.close()
plt.plot(np.abs(specL.sum(axis=0)),rasterized=True)
plt.savefig('hsrFig/specSum.jpg',dpi=300)


In [39]:
reload(hsr)
h=hsr.hsr()
line=h.getLine(stations,sta0='PKU28',sta1='SUS47')

In [40]:
#print(stations.loc())
H0=line.h(np.array(stations.loc()).transpose())
L0=line.l(np.array(stations.loc()).transpose())
H=np.abs(H0)
L=L0.copy()
plt.close()
plt.plot(H,L,'.')
plt.savefig('hsrFig/lh.jpg',dpi=300)

In [41]:
sl=1/3
sh=0.01
dtL=sl*L+sh*H
specL=h.ShiftDt(specL,dtL)

In [55]:
reload(hsr)
h=hsr.hsr(fL0=np.arange(5,5.35,0.005))
slL = np.arange(-1.001,1,0.01)
shL = np.arange(-1.001,1,0.01)#np.arange(-1.001,1,0.01)
specL  = h.getSpecL(t3L,1)
M = h.loopSlh(specL,slL,shL,L,H)



/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [62]:
reload(hsr)
h=hsr.hsr(fL0=np.arange(4.5,5.5,0.005))
plt.close()
#print(np.array(M).shape,M[1][1])
#h.plotSlhByF(M,slL,shL,saveDir='hsrFig/slh/')
#h.plotSlhAll(M,slL,shL,saveDir='hsrFig/slh/')
h.plotSlh3D(M,slL,shL,saveDir='hsrFig/slh/')


[149 149 172 172 172 172 171 178 178 197 197  42  42 143  16  16  76  29
 157 102 157 157   1  75  74  47  74  91  22  22  89 181 181 181 153  59
 142 174 185  17 105  53 162  31  64  64  68 115  31 105 105 105 185 172
 178 178 177 177  61  61  61 166 166 148 147 147 147 147 137 137 137  23
  71 181 134  89  89 199 147 114 114 123 123 123 198   3   3  84  84  84
  84 104 104 104  94  34  32  32  32 168 162  33 161  74 141 141  45 152
  66   2  69  69  27  99  76 170  44  68  69 200   7  13  13  13 179  34
  34  34  26  17  17  17  18  34  58  28  99  88  24 184  64  69 157 157
 145 145 145  97  95 176 176 142  34  34  36 168 168 167 106  59  16 106
  64 152 152 198 197 152 198  19  92  12  12 158  82  82 158 157 156 156
 169 156  55  55 122   9   9 138 138   8   8  33  33 196  20   3 196 169
 169  58]


/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/matplotlib/collections.py:923: ComplexWarning: Casting complex values to real discards the imaginary part
  self._transforms[:, 0, 0] = scale
/home/jiangyr/conda/envs/seismToolNew/lib/python3.6/site-packages/matplotlib/collections.py:924: ComplexWarning: Casting complex values to real discards the imaginary part
  self._transforms[:, 1, 1] = scale
